In [1]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.cluster import KMeans

In [2]:
!pip install pandas==1.5.2
!pip install joblib==1.2.0
from joblib import load, dump

In [3]:
tracks = load("dane_do_modelu/tracks_df_preprocessed")

In [4]:
# for clustering
tracks_only_features = tracks[['danceability','key','energy','loudness','speechiness','acousticness','instrumentalness','liveness','valence','tempo']].copy()

In [5]:
# for determining the genre that characterizes the group
tracks_only_genres = tracks[['artists_genre']]

# Czy skalowanie coś da?

In [6]:
from sklearn.metrics import silhouette_score

In [7]:
from sklearn.preprocessing import StandardScaler

In [8]:
scaler = StandardScaler()
scaler.fit(tracks_only_features)
tracks_only_features_scaled = scaler.transform(tracks_only_features)

In [9]:
number_of_clusters = 1000

In [10]:
X = np.array(tracks_only_features).astype(float)
model = KMeans(n_clusters=number_of_clusters)
clusters_for_tracks = model.fit_predict(X)


C:\Users\leras\Documents\IUM\venv\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [11]:
X_scaled = np.array(tracks_only_features_scaled).astype(float)
model = KMeans(n_clusters=number_of_clusters)
clusters_for_tracks_scaled = model.fit_predict(X_scaled)

C:\Users\leras\Documents\IUM\venv\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


# Gatunki grup

In [12]:
artists_genres = tracks_only_genres.values.tolist()

In [13]:
def most_frequent(List):
  return max(set(List), key = List.count)

In [14]:
def find_clusters_genres(clusters_for_tracks):
    genres_per_cluster = []
    for i in range(number_of_clusters):
      genres_per_cluster.append([])

    for i in range(len(clusters_for_tracks)):
      genres = artists_genres[i][0]
      for genre in genres:
        genres_per_cluster[clusters_for_tracks[i]].append(genre)

    clusters_genres = []
    for cluster in genres_per_cluster:
      clusters_genres.append(most_frequent(cluster))
    return clusters_genres

In [15]:
clusters_genres = find_clusters_genres(clusters_for_tracks)
  

In [16]:
clusters_genres_scaled = find_clusters_genres(clusters_for_tracks_scaled)

In [17]:
print(len(set(clusters_genres)))
print(set(clusters_genres))

10
{'ugandan pop', 'easy_listening', 'salsa puertorriquena', 'latin', 'rock', 'r&b', 'pop', 'acoustic', 'traditional_folk', 'jazz'}


In [18]:
print(len(set(clusters_genres_scaled)))
print(set(clusters_genres_scaled))

7
{'easy_listening', 'latin', 'rock', 'traditional_folk', 'pop', 'rap', 'jazz'}


In [19]:
tracks_genres=[]
for cluster in clusters_for_tracks:
  tracks_genres.append(clusters_genres[cluster])
tracks_with_genres = tracks.assign(genre=tracks_genres)

In [20]:
tracks_with_genres.groupby('genre').count()

,id,name,popularity,duration_ms,explicit,id_artist,release_date,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,artists_genre
genre,,,,,,,,,,,,,,,,,,
acoustic,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15
easy_listening,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784
jazz,110,110,110,110,110,110,110,110,110,110,110,110,110,110,110,110,110,110
latin,3343,3343,3343,3343,3343,3343,3343,3343,3343,3343,3343,3343,3343,3343,3343,3343,3343,3343
pop,66271,66271,66271,66271,66271,66271,66271,66271,66271,66271,66271,66271,66271,66271,66271,66271,66271,66271
r&b,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28
rock,11758,11758,11758,11758,11758,11758,11758,11758,11758,11758,11758,11758,11758,11758,11758,11758,11758,11758
salsa puertorriquena,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4
traditional_folk,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75


In [21]:
tracks_genres=[]
for cluster in clusters_for_tracks_scaled:
  tracks_genres.append(clusters_genres_scaled[cluster])
tracks_with_genres_scaled = tracks.assign(genre=tracks_genres)

In [22]:
tracks_with_genres_scaled.groupby('genre').count()

,id,name,popularity,duration_ms,explicit,id_artist,release_date,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,artists_genre
genre,,,,,,,,,,,,,,,,,,
easy_listening,714,714,714,714,714,714,714,714,714,714,714,714,714,714,714,714,714,714
jazz,265,265,265,265,265,265,265,265,265,265,265,265,265,265,265,265,265,265
latin,2926,2926,2926,2926,2926,2926,2926,2926,2926,2926,2926,2926,2926,2926,2926,2926,2926,2926
pop,66300,66300,66300,66300,66300,66300,66300,66300,66300,66300,66300,66300,66300,66300,66300,66300,66300,66300
rap,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62
rock,11917,11917,11917,11917,11917,11917,11917,11917,11917,11917,11917,11917,11917,11917,11917,11917,11917,11917
traditional_folk,209,209,209,209,209,209,209,209,209,209,209,209,209,209,209,209,209,209


# Jakość?

In [23]:
silhouette_avg = silhouette_score(X, clusters_for_tracks)

In [24]:
silhouette_avg_scaled = silhouette_score(X_scaled, clusters_for_tracks_scaled)

In [25]:
print(silhouette_avg)
print(silhouette_avg_scaled)

0.2207341409019833
0.09240314873643636


In [26]:
#save data
dump(tracks_with_genres, 'dane_do_klasyfikacji/tracks_genres_df')
dump(tracks_with_genres_scaled, 'dane_do_klasyfikacji/tracks_genres_scaled_df')

['dane_do_klasyfikacji/tracks_genres_scaled_df']